In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

# from folium import LayerControl
from eo_tools_dev.util import show_insar_coh, show_insar_phi, show_sar_int
import geopandas as gpd
from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

# Set up parameters and output dir

## Important
- The products must be **unzipped** either manually or by using `extract=True` in the download call. 
- In the following products, only IW1 and IW2 are relevant as IW3 contains only water pixels.

In [ ]:
# change to your custom locations
data_dir = "/data/S1"

ids = ['S1A_IW_SLC__1SDV_20181228T050448_20181228T050515_025221_02C9BE_1E22',
 'S1B_IW_SLC__1SDV_20181222T050400_20181222T050427_014150_01A4AE_4A45']
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
outputs_prefix="/data/res/dev-partial-overlap"
# 

# Download S-1 products

In [ ]:
# load a geometry
# file_aoi = "../data/Etna.geojson"
# shp = gpd.read_file(file_aoi).geometry[0]

# search_criteria = {
#     "productType": "S1_SAR_SLC",
#     "start": "2018-12-22",
#     "end": "2018-12-29",
#     "geom": shp
# }

# results, _ = dag.search(**search_criteria)

In [ ]:
# from eo_tools.util import explore_products
# explore_products(results, shp)

In [ ]:
# indices_to_download = [2, 5]
# to_dl = [results[i] for i in indices_to_download]
# dag.download_all(to_dl, outputs_prefix="/data/S1/", extract=False)

In [ ]:
from eo_tools.S1.core import S1IWSwath
from eo_tools.auxils import get_burst_geometry
from eo_tools.util import _has_overlap
# prm = S1IWSwath(primary_dir, iw=1, pol='vv')
# sec = S1IWSwath(secondary_dir, iw=1, pol='vv')

gdf_burst_prm = get_burst_geometry(
    primary_dir, target_subswaths=["IW1"], polarization="VV"
)
gdf_burst_sec = get_burst_geometry(
    secondary_dir, target_subswaths=["IW1"], polarization="VV"
)


In [ ]:
gdf_burst_prm
# gdf_burst_sec

In [ ]:
# gdf_burst_prm
gdf_burst_sec

In [ ]:
from folium import Map
m = Map()
gdf_burst_prm.explore(m=m)
gdf_burst_sec.explore(m=m)

# Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import process_insar

out_dir = process_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=True,
    write_secondary_amplitude=False,
    apply_fast_esd=True,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 10],
    filter_ifg=True,
    multilook=[1, 4],
    warp_kernel="bicubic",
    clip_to_shape=True,
)

In [ ]:
show_insar_coh(f"{out_dir}/coh_vv.tif")

In [ ]:
show_insar_phi(f"{out_dir}/phi_vv.tif")

In [ ]:
show_sar_int(f"{out_dir}/amp_prm_vv.tif")